### CDC Data Extracion

Extraction of the CDC data from the CDC webtool CDC Wonder using website automation tool, Selenium, to generate text file downloads of the CDC Natality Birth data from their three datasets. 

In [1]:
# Data Extraction
# CDC Natality Birth Rate Data
# Version 14
## 1/26/25
## 19:30

In [2]:
import sys
import os

# Add the root directory to sys.path
sys.path.append(os.path.abspath(".."))

# Now you can use absolute imports
from d497_helpers import config

import platform
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

from selenium.webdriver import Chrome 

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
#import math
#import csv
import re
import time
import pandas as pd
import numpy as np
import os
import shutil
import datetime
import sys

import functools

    

# ORIGINAL AUTHOR CREDIT:
    # author = "Theodore L Caputi",
    # author_email = "tcaputi@gmail.com",
    # url="https://github.com/tlcaputi/cdcwonderpy",
    # license = "No License",




def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)






class wonder:     

    def natality(self,
            #DATABASE,
            RUN_NAME,
            by_variables,
            YEARG,
            MONTHG,
            WEEKDAYG,
            GENDERG,
            STATEG,
            ): 


        #print(type(YEARG))
        #print(YEARG)
        
        get_year = self.YEARG
        #print(get_year)

        
        # Database Index To Select Between the different databases in the xpath filters below
            # 0 == 2007-2023 (DEFAULT)
            # 1 == 2003-2006
            # 2 == 1995-2002

        DATABASE_Index = 0

        #service = Service(executable_path=(config.global_bin + "/chromedriver.exe")

        chrome_options = webdriver.ChromeOptions()

        if self.download_dir is not None:
            download_dir = self.download_dir
        else:
            download_dir = config.global_raw_cdc_data_folder_path

        chrome_options.add_experimental_option("prefs", {
            "download.default_directory": download_dir,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
            })

        #d = Chrome(service = service, options=chrome_options)
        d = Chrome(options=chrome_options)

        d.implicitly_wait(30)

        DB_URL = ""

        # base_url = "https://wonder.cdc.gov/ucd-icd10.html"
        #base_url = "https://wonder.cdc.gov/mcd-icd10.html"

        # base_url = "https://wonder.cdc.gov/controller/datarequest/D66"
        base_url = "http://wonder.cdc.gov/natality-current.html"

        landing_url = "https://wonder.cdc.gov/natality.html"

        DB_2007_2003_EXPANDED_PROVISIONAL_url = "http://wonder.cdc.gov/natality-expanded-provisional.html"
        DB_2007_2003_EXPANDED_url = "http://wonder.cdc.gov/natality-expanded-current.html"
        DB_2007_2023_url = "http://wonder.cdc.gov/natality-current.html"
        DB_2003_2006_url = "http://wonder.cdc.gov/natality-v2006.html"
        DB_1995_2002_url = "http://wonder.cdc.gov/natality-v2002.html"

        
        if get_year >= 2007:
            #print("IF YEARG - Database - 2007_2023")
            #DB_URL = DB_2007_2023_url
            DB_URL = "http://wonder.cdc.gov/natality-current.html"
            DATABASE_Index = 0

        elif get_year >= 2003:
            #print("ELIF YEARG - Database - 2003_2006")
            #DB_URL = DB_2003_2006_url
            DB_URL = "http://wonder.cdc.gov/natality-v2006.html"
            DATABASE_Index = 1
           
        elif get_year >= 1995:
            #print("ELIF YEARG - Database - 1995_2002")
            #DB_URL == DB_1995_2002_url
            DB_URL = "http://wonder.cdc.gov/natality-v2002.html"
            DATABASE_Index = 2

        else:
            #print("ELSE YEARG - Database - DEFAULTING - 2007_2003")
            #DB_URL = base_url
            DB_URL = "http://wonder.cdc.gov/natality-current.html"
            DATABASE_Index = 0



        # if DATABASE == "DB_2007_2023": 
        #     print("IF DB - Database - 2007_2023")
        #     DB_URL = DB_2007_2023_url
        #     DATABASE_Index = 0

        # elif DATABASE == "DB_2003_2006":
        #     print("ELIF DB - Database - 2003_2006")
        #     DB_URL = DB_2003_2006_url
        #     DATABASE_Index = 1

        # elif DATABASE == "DB_1995-2002":
        #     print("ELIF DB - Database - 1995_2002")
        #     DB_URL == DB_1995_2002_url
        #     DATABASE_Index = 2

        # else:
        #     print("ELSE DB - Database - DEFAULTING - 2007_2003")
        #     DB_URL = base_url 
        #     DATABASE_Index = 0

        #d.get(base_url)
        d.get(DB_URL)

        #accept_button = d.find_element("name", "button2")[1]
        accept_button = d.find_element("name", "action-I Agree")
        d.execute_script("arguments[0].click();", accept_button)
        # d.execute_script("sendTab();")
        # accept_button.click()


    # Organize Table Layout 
        
        ## Group Results By 
 
        if (get_year < 2003) & (get_year >= 1995):
            ct = 1
            for var in by_variables:
                if var == 'state':
                    ## By State
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Maternal residence']/option[text()='State']".format(ct)).click()
                    d.find_element(By.XPATH, "//select[@name='B_{}']/optgroup[@label='Maternal residence']/option[text()='State']".format(ct)).click()
                elif var == 'county':
                    ## By County
                    #d.find_elment_by_xpath("//select[@name='B_{}']/optgroup[@label='Maternal residence']/option[text()='County']".format(ct)).click()
                    #d.find_elment_by_xpath("//select[@name='B_{}']/optgroup[@label='Maternal residence']/option[text()='County']".format(ct)).click()
                    d.find_element(By.XPATH, "//select[@name='B_{}']/optgroup[@label='Maternal residence']/option[text()='County']".format(ct)).click()
                elif var == 'year':
                    ## By Year
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Year and Month']/option[text()='Year']".format(ct)).click()
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Year']".format(ct)).click()
                    d.find_element(By.XPATH, "//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Year']".format(ct)).click()
                elif var == 'month':
                    ## By Month
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Year and Month']/option[text()='Month']".format(ct)).click()
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Month']".format(ct)).click()
                    #d.find_element(By.XPATH, "//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Month']".format(ct)).click()
                    ct + 1
                #elif var == "age":
                    ## By Age Groups
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Age Groups']".format(ct)).click()
                elif var == "weekday":
                    ## By Weekday
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Weekday']".format(ct)).click()
                    #d.find_element(By.XPATH, "//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Weekday']".format(ct)).click()
                    ct + 1
                elif var == "gender":
                    ## By Baby Gender
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Age Groups']".format(ct)).click()
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Gender']".format(ct)).click()
                    d.find_element(By.XPATH, "//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Gender']".format(ct)).click()
                #elif var == "race":
                    # ## By Race
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Race']".format(ct)).click()
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Race']".format(ct)).click()
                #elif var == "hispanic":
                    ## By Hispanic
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Hispanic Origin']".format(ct)).click()
                #elif var == "gender" or var == "sex":
                    ## By Gender
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Gender']".format(ct)).click()
                elif var == '':
                    ct = ct - 1

                ct = ct + 1
        else: 
            ct = 1
            for var in by_variables:
                if var == 'state':
                    ## By State
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Maternal residence']/option[text()='State']".format(ct)).click()
                    d.find_element(By.XPATH, "//select[@name='B_{}']/optgroup[@label='Maternal residence']/option[text()='State']".format(ct)).click()
                elif var == 'county':
                    ## By County
                    #d.find_elment_by_xpath("//select[@name='B_{}']/optgroup[@label='Maternal residence']/option[text()='County']".format(ct)).click()
                    #d.find_elment_by_xpath("//select[@name='B_{}']/optgroup[@label='Maternal residence']/option[text()='County']".format(ct)).click()
                    d.find_element(By.XPATH, "//select[@name='B_{}']/optgroup[@label='Maternal residence']/option[text()='County']".format(ct)).click()
                elif var == 'year':
                    ## By Year
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Year and Month']/option[text()='Year']".format(ct)).click()
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Year']".format(ct)).click()
                    d.find_element(By.XPATH, "//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Year']".format(ct)).click()
                elif var == 'month':
                    ## By Month
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Year and Month']/option[text()='Month']".format(ct)).click()
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Month']".format(ct)).click()
                    d.find_element(By.XPATH, "//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Month']".format(ct)).click()
                #elif var == "age":
                    ## By Age Groups
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Age Groups']".format(ct)).click()
                elif var == "weekday":
                    ## By Weekday
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Weekday']".format(ct)).click()
                    d.find_element(By.XPATH, "//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Weekday']".format(ct)).click()
                elif var == "gender":
                    ## By Baby Gender
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Age Groups']".format(ct)).click()
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Gender']".format(ct)).click()
                    d.find_element(By.XPATH, "//select[@name='B_{}']/optgroup[@label='Birth characteristics']/option[text()='Gender']".format(ct)).click()
                #elif var == "race":
                    # ## By Race
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Race']".format(ct)).click()
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Race']".format(ct)).click()
                #elif var == "hispanic":
                    ## By Hispanic
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Hispanic Origin']".format(ct)).click()
                #elif var == "gender" or var == "sex":
                    ## By Gender
                    #d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Gender']".format(ct)).click()
                elif var == '':
                    ct = ct - 1

                ct = ct + 1




        ## Run name
        #d.find_element_by_css_selector("input#TO_title").send_keys(RUN_NAME)
        d.find_element(By.CSS_SELECTOR, "input#TO_title").send_keys(RUN_NAME)
        
        
        #formatted_run_name += "-".join(RUN_NAME, YEARG)
        #d.find_element(By.CSS_SELECTOR, "input#TO_title").send_keys(formatted_run_name)

        def double_click(element):
            actionChains = ActionChains(d)
            actionChains.double_click(element).perform()

        def move_to_element(element):
            action = ActionChains(d)
            action.move_to_element(element).click().perform()

        def control_click(element):
            action = ActionChains(d)
            action.key_down(Keys.CONTROL).click(element).key_up(Keys.CONTROL).perform()


        def scroll_element_into_view(element):
            d.execute_script('arguments[0].scrollIntoView(true)', element);
            time.sleep(3);
            d.execute_script('window.scrollBy(0,-50)')
            time.sleep(3)


        def move_arrow_up(element):
            return 
            



        ## Birth Characteristics

        ### Years
            # 2007 - 2023 -- V_D66.V20
            # 2003 - 2006 -- V_D27.V20
            # 1995 - 2002 -- V_D10.V20



        if DATABASE_Index == 0:

            if self.YEARG is not None:
                #print("not none")
                if self.YEARG != 2023:
                    #print("not 2023")

                    if (2023 - self.YEARG) < 3:

                        #print("year differnce is less than 3")
                        control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='2019']"))
                        time.sleep(5)
                        scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='2019']"))
                        time.sleep(3)
                        control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='2023']"))
                        time.sleep(5)           
                        scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='2023']"))
                        time.sleep(3)
                        control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='{}']".format(self.YEARG)))
                        time.sleep(5)
                        
                    else:
                        #print("year differnce is 3 or more")
                        #print("scolling")
                        scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='{}']".format(self.YEARG)))
                        time.sleep(3)
                        #print("control click")
                        control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='{}']".format(self.YEARG)))
                        time.sleep(5)
                        #control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='{}']".format(self.YEARG)))
                        #time.sleep(5)
                        #scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='{}']".format(self.YEARG)))
                        #time.sleep(3)
                        #control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='{}']".format(self.YEARG)))
                        #time.sleep(10)
                        #scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V20']"))
                        #time.sleep(3)
                        #print("move to element")
                        #move_to_element(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='2023']"))
                        #time.sleep(10)
                        #print("scrolling")
                        scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='2023']"))
                        time.sleep(3)
                        #print("control click")
                        control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='2023']"))
                        time.sleep(5)

                else:
                    #print("2023")
                    scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V20']"))
                    time.sleep(3)
                    double_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='{}']".format(self.YEARG)))          

                

            ### Months
                # 2007 - 2023 -- V_D66.V25
                # 2003 - 2006 -- V_D27.V25
                # 1995 - 2002 -- V_D10.V25

            if self.MONTHG is not None:
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V25']/option[@value='{}']".format(self.MONTHG)))
                time.sleep(5)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V25']/option[@value='{}']".format(self.MONTHG)))
                time.sleep(3)
                #control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V25']/option[@value='{}']".format(self.MONTHG)))
                #time.sleep(10)
                move_to_element(d.find_element(By.XPATH, "//select[@name='V_D66.V25']/option[@value='*All*']"))
                time.sleep(10)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V25']/option[@value='{}']".format(self.MONTHG)))
                time.sleep(3)
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V25']/option[@value='*All*']"))
                time.sleep(5)


            ### Weekday
                # 2007 - 2023 -- V_D66.V26
                # 2003 - 2006 -- V_D27.V26
                # 1995 - 2002 -- V_D10.V26

            if self.WEEKDAYG is not None:
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V26']/option[@value='{}']".format(self.WEEKDAYG)))
                time.sleep(5)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V26']/option[@value='{}']".format(self.WEEKDAYG)))
                time.sleep(3)
                #control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V26']/option[@value='{}']".format(self.WEEKDAYG)))
                #time.sleep(10)
                move_to_element(d.find_element(By.XPATH, "//select[@name='V_D66.V26']/option[@value='*All*']"))
                time.sleep(5)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V26']/option[@value='{}']".format(self.WEEKDAYG)))
                time.sleep(3)
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V26']/option[@value='*All*']"))
                time.sleep(10)

            ### Gender
                # 2007 - 2023 -- V_D66.V3
                # 2003 - 2006 -- V_D27.V3
                # 1995 - 2002 -- V_D10.V21

            if self.GENDERG is not None:
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V3']/option[@value='{}']".format(self.GENDERG)))
                time.sleep(3)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V3']/option[@value='{}']".format(self.GENDERG)))
                time.sleep(3)
                #control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V3']/option[@value='{}']".format(self.GENDERG)))
                #time.sleep(5)
                move_to_element(d.find_element(By.XPATH, "//select[@name='V_D66.V3']/option[@value='*All*']"))
                time.sleep(5)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V3']/option[@value='{}']".format(self.GENDERG)))
                time.sleep(3)
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V3']/option[@value='*All*']"))
                time.sleep(3)

            ## Maternal Residence

                ### State
                # 2007 - 2023 -- F_D66.V21
                # 2003 - 2006 -- F_D27.V21
                # 1995 - 2002 -- F_D10.V21

            if self.STATEG is not None:
                # This one most likely does not work or needs the same methods as above but in loop for a list of states.
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V21']/option[@value='{}']".format(self.STATEG)))
                time.sleep(5)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V21']/option[@value='{}']".format(self.STATEG)))
                time.sleep(3)
                #control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V21']/option[@value='{}']".format(self.STATEG)))
                #time.sleep(10)
                move_to_element(d.find_element(By.XPATH, "//select[@name='V_D66.V21']/option[@value='*All*']"))
                time.sleep(10)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V21']/option[@value='{}']".format(self.STATEG)))
                time.sleep(3)
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V21']/option[@value='*All*']"))
                time.sleep(5)  

        elif DATABASE_Index == 1:

            #print("In database_index 1")

            if self.YEARG is not None:
                #d.find_element_by_xpath("//select[@name='V_D66.V20']").send_keys(Keys.CONTROL, 'a')
                #double_click(d.find_element_by_xpath("//select[@name='V_D66.V20']/option[@value='{}']".format(self.YEARG)))
                d.find_element(By.XPATH, "//select[@name='V_D27.V20']").send_keys(Keys.CONTROL, 'a')
                double_click(d.find_element(By.XPATH, "//select[@name='V_D27.V20']/option[@value='{}']".format(self.YEARG)))

            ### Months
                # 2007 - 2023 -- V_D66.V25
                # 2003 - 2006 -- V_D27.V25
                # 1995 - 2002 -- V_D10.V25

            if self.MONTHG is not None:
                #d.find_element_by_xpath("//select[@name='V_D66.V25']").send_keys(Keys.CONTROL, 'a')
                #double_click(d.find_element_by_xpath("//select[@name='V_D66.V25']/option[@value='{}']".format(self.MONTHG)))          
                d.find_element(By.XPATH, "//select[@name='V_D27.V25']").send_keys(Keys.CONTROL, 'a')
                double_click(d.find_element(By.XPATH, "//select[@name='V_D27.V25']/option[@value='{}']".format(self.MONTHG)))          


            ### Weekday
                # 2007 - 2023 -- V_D66.V26
                # 2003 - 2006 -- V_D27.V26
                # 1995 - 2002 -- V_D10.V26

            if self.WEEKDAYG is not None:
                #d.find_element_by_xpath("//select[@name='V_D66.V26']").send_keys(Keys.CONTROL, 'a')
                #double_click(d.find_element_by_xpath("//select[@name='V_D66.V26']/option[@value='{}']".format(self.WEEKDAYG))) 
                d.find_element(By.XPATH, "//select[@name='V_D27.V26']").send_keys(Keys.CONTROL, 'a')
                double_click(d.find_element(By.XPATH, "//select[@name='V_D27.V26']/option[@value='{}']".format(self.WEEKDAYG))) 

            ### Gender
                # 2007 - 2023 -- V_D66.V3
                # 2003 - 2006 -- V_D27.V3
                # 1995 - 2002 -- V_D10.V21

            if self.GENDERG is not None:
                #d.find_element_by_xpath("//select[@name='V_D66.V3']").send_keys(Keys.CONTROL, 'a')
                #double_click(d.find_element_by_xpath("//select[@name='V_D66.V3']/option[@value='{}']".format(self.GENDERG))) 
                d.find_element(By.XPATH, "//select[@name='V_D27.V3']").send_keys(Keys.CONTROL, 'a')
                double_click(d.find_element(By.XPATH, "//select[@name='V_D27.V3']/option[@value='{}']".format(self.GENDERG))) 


            ## Maternal Residence

                ### State
                # 2007 - 2023 -- F_D66.V21
                # 2003 - 2006 -- F_D27.V21
                # 1995 - 2002 -- F_D10.V21

            if self.STATEG is not None:
                #d.find_element_by_xpath("//select[@name='F_D66.V21']").send_keys(Keys.CONTROL, 'a')
                #double_click(d.find_element_by_xpath("//select[@name='F_D66.V21']/option[@value='{}']".format(self.STATEG)))            
                d.find_element(By.XPATH, "//select[@name='F_D27.V21']").send_keys(Keys.CONTROL, 'a')
                double_click(d.find_element(By.XPATH, "//select[@name='F_D27.V21']/option[@value='{}']".format(self.STATEG)))  


        elif DATABASE_Index == 2:

            #print("In database_index 2")

            if self.YEARG is not None:
                #d.find_element_by_xpath("//select[@name='V_D66.V20']").send_keys(Keys.CONTROL, 'a')
                #double_click(d.find_element_by_xpath("//select[@name='V_D66.V20']/option[@value='{}']".format(self.YEARG)))
                d.find_element(By.XPATH, "//select[@name='V_D10.V20']").send_keys(Keys.CONTROL, 'a')
                double_click(d.find_element(By.XPATH, "//select[@name='V_D10.V20']/option[@value='{}']".format(self.YEARG)))

            ### Months
                # 2007 - 2023 -- V_D66.V25
                # 2003 - 2006 -- V_D27.V25
                # 1995 - 2002 -- V_D10.V25

            if self.MONTHG is not None:
                #d.find_element_by_xpath("//select[@name='V_D66.V25']").send_keys(Keys.CONTROL, 'a')
                #double_click(d.find_element_by_xpath("//select[@name='V_D66.V25']/option[@value='{}']".format(self.MONTHG)))          
                d.find_element(By.XPATH, "//select[@name='V_D10.V25']").send_keys(Keys.CONTROL, 'a')
                double_click(d.find_element(By.XPATH, "//select[@name='V_D10.V25']/option[@value='{}']".format(self.MONTHG)))          


            ### Weekday
                # 2007 - 2023 -- V_D66.V26
                # 2003 - 2006 -- V_D27.V26
                # 1995 - 2002 -- V_D10.V26

            if self.WEEKDAYG is not None:
                #d.find_element_by_xpath("//select[@name='V_D66.V26']").send_keys(Keys.CONTROL, 'a')
                #double_click(d.find_element_by_xpath("//select[@name='V_D66.V26']/option[@value='{}']".format(self.WEEKDAYG))) 
                d.find_element(By.XPATH, "//select[@name='V_D10.V26']").send_keys(Keys.CONTROL, 'a')
                double_click(d.find_element(By.XPATH, "//select[@name='V_D10.V26']/option[@value='{}']".format(self.WEEKDAYG))) 

            ### Gender
                # 2007 - 2023 -- V_D66.V3
                # 2003 - 2006 -- V_D27.V3
                # 1995 - 2002 -- V_D10.V21

            if self.GENDERG is not None:
                #d.find_element_by_xpath("//select[@name='V_D66.V3']").send_keys(Keys.CONTROL, 'a')
                #double_click(d.find_element_by_xpath("//select[@name='V_D66.V3']/option[@value='{}']".format(self.GENDERG))) 
                d.find_element(By.XPATH, "//select[@name='V_D10.V3']").send_keys(Keys.CONTROL, 'a')
                double_click(d.find_element(By.XPATH, "//select[@name='V_D10.V3']/option[@value='{}']".format(self.GENDERG))) 


            ## Maternal Residence

                ### State
                # 2007 - 2023 -- F_D66.V21
                # 2003 - 2006 -- F_D27.V21
                # 1995 - 2002 -- F_D10.V21

            if self.STATEG is not None:
                #d.find_element_by_xpath("//select[@name='F_D66.V21']").send_keys(Keys.CONTROL, 'a')
                #double_click(d.find_element_by_xpath("//select[@name='F_D66.V21']/option[@value='{}']".format(self.STATEG)))            
                d.find_element(By.XPATH, "//select[@name='F_D10.V21']").send_keys(Keys.CONTROL, 'a')
                double_click(d.find_element(By.XPATH, "//select[@name='F_D10.V21']/option[@value='{}']".format(self.STATEG)))  

        else: 

            if self.YEARG is not None:
                
                if self.YEARG != 2023:
                    
                    year_value = int(self.YEARG)
                    year_difference = (2023 - year_value)
                        
                    if (year_difference - self.YEARG) < 2:
                        
                        control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='2019']"))
                        time.sleep(5)
                        scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='2019']"))
                        time.sleep(3)
                        control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='2023']"))
                        time.sleep(5)           
                        scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='2023']"))
                        time.sleep(3)
                        control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='{}']".format(self.YEARG)))
                        time.sleep(5)
                        
                    else:
                        control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='{}']".format(self.YEARG)))
                        time.sleep(5)
                        scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V20']"))
                        time.sleep(3)
                        #control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='{}']".format(self.YEARG)))
                        #time.sleep(10)
                        #scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V20']"))
                        #time.sleep(3)
                        #move_to_element(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='2023']"))
                        #time.sleep(10)
                        scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V20']"))
                        time.sleep(3)
                        control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='2023']"))
                        time.sleep(5)
            
            else: 
                    double_click(d.find_element(By.XPATH, "//select[@name='V_D66.V20']/option[@value='{}']".format(self.YEARG))) 

                

            ### Months
                # 2007 - 2023 -- V_D66.V25
                # 2003 - 2006 -- V_D27.V25
                # 1995 - 2002 -- V_D10.V25

            if self.MONTHG is not None:
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V25']/option[@value='{}']".format(self.MONTHG)))
                time.sleep(5)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V25']/option[@value='{}']".format(self.MONTHG)))
                time.sleep(3)
                #control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V25']/option[@value='{}']".format(self.MONTHG)))
                #time.sleep(10)
                move_to_element(d.find_element(By.XPATH, "//select[@name='V_D66.V25']/option[@value='*All*']"))
                time.sleep(10)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V25']/option[@value='{}']".format(self.MONTHG)))
                time.sleep(3)
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V25']/option[@value='*All*']"))
                time.sleep(5)


            ### Weekday
                # 2007 - 2023 -- V_D66.V26
                # 2003 - 2006 -- V_D27.V26
                # 1995 - 2002 -- V_D10.V26

            if self.WEEKDAYG is not None:
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V26']/option[@value='{}']".format(self.WEEKDAYG)))
                time.sleep(5)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V26']/option[@value='{}']".format(self.WEEKDAYG)))
                time.sleep(3)
                #control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V26']/option[@value='{}']".format(self.WEEKDAYG)))
                #time.sleep(10)
                move_to_element(d.find_element(By.XPATH, "//select[@name='V_D66.V26']/option[@value='*All*']"))
                time.sleep(5)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V26']/option[@value='{}']".format(self.WEEKDAYG)))
                time.sleep(3)
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V26']/option[@value='*All*']"))
                time.sleep(10)

            ### Gender
                # 2007 - 2023 -- V_D66.V3
                # 2003 - 2006 -- V_D27.V3
                # 1995 - 2002 -- V_D10.V21

            if self.GENDERG is not None:
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V3']/option[@value='{}']".format(self.GENDERG)))
                time.sleep(3)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V3']/option[@value='{}']".format(self.GENDERG)))
                time.sleep(3)
                #control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V3']/option[@value='{}']".format(self.GENDERG)))
                #time.sleep(5)
                move_to_element(d.find_element(By.XPATH, "//select[@name='V_D66.V3']/option[@value='*All*']"))
                time.sleep(5)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V3']/option[@value='{}']".format(self.GENDERG)))
                time.sleep(3)
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V3']/option[@value='*All*']"))
                time.sleep(3)


            ## Maternal Residence

                ### State
                # 2007 - 2023 -- F_D66.V21
                # 2003 - 2006 -- F_D27.V21
                # 1995 - 2002 -- F_D10.V21

            if self.STATEG is not None:
                # This one most likely does not work or needs the same methods as above but in loop for a list of states.
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V21']/option[@value='{}']".format(self.STATEG)))
                time.sleep(5)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V21']/option[@value='{}']".format(self.STATEG)))
                time.sleep(3)
                #control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V21']/option[@value='{}']".format(self.STATEG)))
                #time.sleep(10)
                move_to_element(d.find_element(By.XPATH, "//select[@name='V_D66.V21']/option[@value='*All*']"))
                time.sleep(10)
                scroll_element_into_view(d.find_element(By.XPATH, "//select[@name='V_D66.V21']/option[@value='{}']".format(self.STATEG)))
                time.sleep(3)
                control_click(d.find_element(By.XPATH, "//select[@name='V_D66.V21']/option[@value='*All*']"))
                time.sleep(5)  




        ### SUBMIT

        # input("Hit enter...")
        # d.find_element_by_css_selector("input#export-option").click()

        #export = d.find_element_by_css_selector("input#export-option")
        #show_zeros = d.find_element_by_css_selector("input#CO_show_zeros")
        export = d.find_element(By.CSS_SELECTOR, "input#export-option")
        show_zeros = d.find_element(By.CSS_SELECTOR, "input#CO_show_zeros")

        d.execute_script("arguments[0].click();", export)
        d.execute_script("arguments[0].click()", show_zeros)

        # d.find_element_by_xpath("//div[@class='footer-buttons']/input[@value='Send']").click()
        #send = d.find_element_by_xpath("//div[@class='footer-buttons']/input[@value='Send']")
        send = d.find_element(By.XPATH, "//div[@class='footer-buttons']/input[@value='Send']")
        try:
            d.execute_script("arguments[0].click();", send)
        except TimeoutException:
            time.sleep(360)
        except:
            time.sleep(360)

        #time.sleep(300)
        #time.sleep(180)
        #time.sleep(120)
        #time.sleep(60)
        #time.sleep(45)
        #time.sleep(30)
        #time.sleep(15)
        time.sleep(10)


        #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
        #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
        #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

        # <>
        # <table class="response-form">
        # <thead> = Column Names = ['Year','Month', 'State', 'County', 'Births']
        
        # Null/Missing Values in the total's column are labeled as "Missing County"
        
        # Grouped by Year, Then Month, then State, then County

        # Each Row contains 5 cells. 
            # 4 of the 5 cells are 'th' cells, which are the 'Year', 'Month', 'State', and 'County'. Values only. No classes, names, ids, etc. 
            # The 5th cell is a 'td' containing the 'Births'. Values only. No classes, names, ids, etc.  

        # At the end of each State Group, there is final last row that contains the Total Value for the State
            # This row contains 5 cells but the format/coding changes for two of the cell. 
            # Cells 1, 2, 3 are the same, 'th' cells containing the 'Year', 'Month', and 'State'
            # Cell 4 changes to be a 'Total' cell. The cell contains a colspan attribute with "1" and a class called "t". The Value is the string "Total"
            # Cell 5 contains the total number of Births, but now has a class assigned to it called "t". 

        # These Groupings and division of cells continue for the entire month before finishing up with a 'total' row for the entire month.
            # The 'Month' total row is made up of 4 cells but uses the colspan attribute to stetch one call into two. 
                # The cells for 1 and 2 are for the 'Year' and 'Month' and are 'th' cells
                # Cell 3 is also a 'th' cell but contains the colspan attribute assigned to '2' to take up 3rd and 4th cell's width. It also has the class "t" as well. 
                # Cell 5 is a 'td' cell contains the total births value for the month, with the class "t" assigned to it. 
    

        # Again this process repeats for the entire year and then follows the same formula that the Month total row does. 
            # The 'Year' total row is made up of 3 cells this time, using the colspan attribute to stetch one call into three. 
                # The 1st cell is for the 'Year' value, and it is a 'th' cell
                # Cell 2 is where the 'total' label cell is, which is a 'th' cell and  contains the colspan attribute assigned to '3' 
                    # to take up what is technicall the 2nd, 3rd, 4th cell's width. It also has the class "t" as well. 
                # Cell 3rd is a 'td' cell contains the total births value for the year, with the class "t" assigned to it. 

        # The last cell of the table is contains an overall total's cell and the value. 
            # The row contains only two cells. 
                # The 1st cell is the "total' label cell which is a 'th' cell assigned with a colspan attribute assigned to "4", making it the with of cells 1-4. It contains the "t" class
                # the 2nd cell is the total value cell, which is a 'td' assigned with a "t" class. 


        # There is a table Notes, Caveats, Citations, Links, Query details, Query Suggetiosn, and Query Criteria section that follows the table. 

            # Table Notes
                # <table class="response-links noprint">
                    # Probably can skip this one. 

            # Caveats
                # <table class="response-notes caveats">
                    # Contains Two cell rows.
                        # Row 1 is the Title row, 
                            # Has 1 'th' cell with a colspan attrible assigned with "2" and class assigned with "title"
                        # Row 2 and on contain note extracts
                            #Rows have two cells, one "th" and One "td"
                                # 1st cells are the "th" cells that contains the Note type/title (Caveat, Help, and Querty Date)
                                # 2nd Cells are the "td" containing text, links, timestamps etc. 

            # Suggested Citations
                # <table class="reponse-notes">
                    # Contains Two rows.
                        # Row 1 is the Title row, 
                            # Has 1 'th' cell with a colspan attrible assigned with "2" and class assigned with "title"
                        # Row 2 is the Citation
                            # Has only 1 'td' cell, not assigned with any colspan or classes

            # Query Criteria
                # <table class="response-notes query-description">
                        # Row 1 is the Title row, 
                            # Has 1 'th' cell with a colspan attrible assigned with "2" and class assigned with "title"
                        # Row 2 is left blank for spacing/design style/choice
                            # Has two cells, one <th> and one <td>
                        # Row 3 and on contain note extracts
                            #Rows have two cells, one "th" and One "td"
                                # 1st cells are the "th" cells that contains the desription item ('year', 'group by', 'show totals', etc)
                                # 2nd Cells are the "td" containing the associated value. 
                        
        

        #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
        #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
        #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
        


        d.quit()




    def natality_or_load(
            self,
            #DATABASE = "",
            RUN_NAME = "",
            by_variables = None,
            existing_file = False,
            YEARG = "",
            MONTHG = None,
            WEEKDAYG = None,
            GENDERG = None,
            STATEG = None
            ):

        if not existing_file:

            self.natality(
                    #DATABASE = DATABASE,
                    RUN_NAME = RUN_NAME,
                    by_variables = by_variables,
                    YEARG = "",
                    MONTHG = None,
                    WEEKDAYG = None,
                    GENDERG = None,
                    STATEG = None
                    )

        else:
            self.df = pd.read_csv("{}/{}_pull.csv".format(self.download_dir, RUN_NAME))


    def __init__(self,
                #DATABASE = "",
                RUN_NAME="",
                YEARG=None,
                MONTHG=None,
                WEEKDAYG=None,
                GENDERG = None,
                STATEG = None,
                by_variables="",
                existing_file = False,
                download_dir = True,
                just_go=False
                ):



        #self.DATABASE = DATABASE
        self.RUN_NAME = RUN_NAME
        self.by_variables = by_variables
        self.download_dir = download_dir
        self.existing_file = existing_file
        self.YEARG = YEARG
        self.MONTHG = MONTHG
        self.WEEKDAYG = WEEKDAYG
        self.GENDERG = GENDERG
        self.STATEG = STATEG



        if just_go is False:

            self.natality_or_load(
                    #DATABASE = self.DATABASE,
                    RUN_NAME = self.RUN_NAME,
                    by_variables = self.by_variables,
                    existing_file = self.existing_file,
                    YEARG = self.YEARG,
                    MONTHG = self.MONTHG,
                    WEEKDAYG = self.WEEKDAYG,
                    GENDERG = self.GENDERG,
                    STATEG = self.STATEG
                    )

In [3]:
# Main function to run the code

def main():
    
    # Run This
    year_start = 1995
    year_end = 2023
    #print([year for year in range(year_start, year_end+1)])

    for year in range(year_start, year_end+1):

        base_text = "raw_cdc_data"

        run_name_text = "_".join([base_text, str(year)])
    
        wonder(
            ## wonderpy arguments
            #DATABASE = "DB_2003_2006",
            RUN_NAME = run_name_text, # Name of the run
            download_dir = None,
            #download_dir = "D:/WGU/D497/Project_1-Real_World_Data_Wrangling_With_Python/file_downloads/", # Where to download the data
            #download_dir = "./file-downloads/", # Where to download the data
            existing_file = False, # True if the data file already exists on your local computer
        
            ## You want deaths matching the following ICD Codes
            #MCD_CAUSE_CODE_1 = ["A10", "A20"], # Deaths that include ANY of these ICD Codes
            #MCD_CAUSE_CODE_2 = ["A12", "A22"], # ...and ANY of these ICD Codes
            
            ## Group the deaths by these variables
            by_variables = ["year", "month", "state", "county"], # Any of the following: "sex", "age", "race", "hispanic", "state", "year", "month"
            
            ## Only collect data of the following (None if all)
            YEARG = year, 
            MONTHG = None,
            WEEKDAYG = None,
            GENDERG = None, 
            STATEG = None, 
            
            ## Other
            just_go = False # For use with more further and undeveloped cases. Keep as False.
        )

    

In [4]:
if __name__ == "__main__":
    main()

The chromedriver version (131.0.6778.87) detected in PATH at C:\WebDriver\bin\chromedriver.exe might not be compatible with the detected chrome version (132.0.6834.112); currently, chromedriver 132.0.6834.159 is recommended for chrome 132.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (131.0.6778.87) detected in PATH at C:\WebDriver\bin\chromedriver.exe might not be compatible with the detected chrome version (132.0.6834.112); currently, chromedriver 132.0.6834.159 is recommended for chrome 132.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (131.0.6778.87) detected in PATH at C:\WebDriver\bin\chromedriver.exe might not be compatible with the detected chrome version (132.0.6834.112); currently, chromedriver 132.0.6834.159 is recommended for chrome 132.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (131.0.6778.87) detected in PATH at C:\WebDriver\bin\chromedriver.exe might not b

## [Next Step: FIPS Data Cleaning](data_cleaning_fips_main.ipynb)

---
#### [Return To Landing Page](order_of_operations_landing.ipynb)